# Constructing symmetric tensor with non-abelian symmetry using CG coefficients directly
The tensor in question is 
$$A[s, s_1,s_2,\cdots, s_6] = \delta(s_1+s_2+\cdots+s_6 - s) f(s)$$
where $f(s)$ is some even function

In [1]:
using TensorKit
include("./aklt3d.jl")
using Einsum

In [3]:
# AKLT center without the Xs on the legs
function aklt3d_center_no_X(a0,a1,a2)
    V = ℝ^2
    W = ℝ^7
    f=Dict(0=>1,1=>a2,2=>a1,3=>a0)
    M = TensorMap(zeros,W,V⊗V⊗V⊗V⊗V⊗V);
    for s1 in 1:2
        for s2 in 1:2 
            for s3 in 1:2
                for S in 1:7
                    for s4 in 1:2
                        for s5 in 1:2
                            for s6 in 1:2 
                                # Now si -> -3/2 + si ; S-> S-4
                                if (s1 + s2 + s3 + s4 + s5 + s6 - 5) == S
                                    M[S,s1,s2,s3,s4,s5,s6] = f[abs(S-4)]
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    return M
end

aklt3d_center_no_X (generic function with 1 method)

In [4]:
V = CU1Space((1/2,2)=>1) # Physical space 
W = CU1Space((0,0)=>1,(1,2)=>1,(2,2)=>1,(3,2)=>1) # Virtual space

Rep[CU₁]((0, 0)=>1, (1, 2)=>1, (2, 2)=>1, (3, 2)=>1)

In [9]:
TT = aklt3d_center_no_X(rand(),rand(),rand()); # Initialize the tensor without symmetry
TT = convert(Array,TT)

7×2×2×2×2×2×2 Array{Float64, 7}:
[:, :, 1, 1, 1, 1, 1] =
 0.39697  0.0
 0.0      0.288504
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.0

[:, :, 2, 1, 1, 1, 1] =
 0.0       0.0
 0.288504  0.0
 0.0       0.396457
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 1, 2, 1, 1, 1] =
 0.0       0.0
 0.288504  0.0
 0.0       0.396457
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 2, 2, 1, 1, 1] =
 0.0       0.0
 0.0       0.0
 0.396457  0.0
 0.0       1.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 1, 1, 2, 1, 1] =
 0.0       0.0
 0.288504  0.0
 0.0       0.396457
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 2, 1, 2, 1, 1] =
 0.0       0.0
 0.0       0.0
 0.396457  0.0
 0.0       1.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 1, 2, 2, 1, 1] =
 0.0       0.0
 0.0       0.0
 0.396457  0.0
 0.0       1.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 2, 2, 2, 1, 1] =
 0.0  0.0
 0.0  0.0


In [10]:
T = TensorMap(zeros,Float64,W,V⊗V⊗V⊗V⊗V⊗V) # Projecting the tensor onto the symmetric subspace
for (f1,f2) in fusiontrees(T)
    coupled_space = f1.coupled
    cg_tensor = convert(Array,f2)
    j = convert(Int,coupled_space.j)
    TT_sector = TT[4+j,1:end,1:end,1:end,1:end,1:end,1:end]
    #@show size(cg_tensor)
    #if j != 0
        #show cg_tensor[1:end,1:end,1:end,1:end,1:end,1:end,1] .- cg_tensor[1:end,1:end,1:end,1:end,1:end,1:end,2]
    #end
    
    if j != 0
        @einsum sector_value := TT_sector[a,b,c,d,e,f]*cg_tensor[a,b,c,d,e,f,2] # I don't know why, but one of the CG coefficients seems to be zero 
    else
        @einsum sector_value := TT_sector[a,b,c,d,e,f]*cg_tensor[a,b,c,d,e,f,1]
    end
    T[f1,f2] .= sector_value
end

In [11]:
AA = convert(Array,T) # Convert symmetric tensor back to the one without symmetry to check correctness


7×2×2×2×2×2×2 Array{Float64, 7}:
[:, :, 1, 1, 1, 1, 1] =
 0.0      0.0
 0.0      0.0
 0.0      0.0
 0.0      0.288504
 0.0      0.0
 0.39697  0.0
 0.0      0.0

[:, :, 2, 1, 1, 1, 1] =
 0.0       0.0
 0.0       0.396457
 0.0       0.0
 0.288504  0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 1, 2, 1, 1, 1] =
 0.0       0.0
 0.0       0.396457
 0.0       0.0
 0.288504  0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 2, 2, 1, 1, 1] =
 0.0       1.0
 0.396457  0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 1, 1, 2, 1, 1] =
 0.0       0.0
 0.0       0.396457
 0.0       0.0
 0.288504  0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 2, 1, 2, 1, 1] =
 0.0       1.0
 0.396457  0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 1, 2, 2, 1, 1] =
 0.0       1.0
 0.396457  0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0
 0.0       0.0

[:, :, 2, 2, 2, 1, 1] =
 1.0  0.0
 0.0  0.0


In [13]:
# TT legs to AA legs, the AA legs are ordered according to irreps, which follows that of W = (0,0)=>1,(1,2)=>1,(2,2)=>1,(3,2)=>1
legs_map = Dict(4=>1,3=>2,5=>3,2=>4,6=>5,1=>6,7=>7) 
for (TT_leg, AA_leg) in legs_map
    abs_diff = sum(abs.(TT[TT_leg,1:end,1:end,1:end,1:end,1:end,1:end,] .- AA[AA_leg,1:end,1:end,1:end,1:end,1:end,1:end,]))
    @show abs_diff
end

abs_diff = 1.2212453270876722e-15
abs_diff = 9.769962616701378e-15
abs_diff = 1.1102230246251565e-16
abs_diff = 0.0
abs_diff = 1.1102230246251565e-16
abs_diff = 1.2212453270876722e-15
abs_diff = 0.0
